In [1]:
# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from time import time

# Metrics
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import time
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score

In [3]:
train = pd.read_csv('/kaggle/input/Cascade_cup/train_age_dataset.csv')
test = pd.read_csv('/kaggle/input/Cascade_cup/test_age_dataset.csv')
sample = pd.read_csv('/kaggle/input/Cascade_cup/sample_submission.csv')

In [4]:
train

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
0,265153,48958844,2,1,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,0.463330,34.160000,53.996091,0,0.000000,0.200000,0.000000,0.041667,0.025000,0.000000,0.000000,0.175000,0.033333,0.000000,1
1,405231,51100441,2,2,0.000000,0.000,0.00,0,0.0,0.0,0.076923,0.153846,0.429468,15.285714,96.231625,0,0.008475,0.093220,0.000000,0.012712,0.018644,0.000000,0.084746,0.000000,0.033898,82.500000,2
2,57867,6887426,2,1,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,0.341657,22.000000,83.051415,0,0.000000,0.002786,0.000000,0.000000,0.000557,0.000000,0.002786,0.000000,0.000000,0.000000,1
3,272618,50742404,2,1,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,0.005735,3.000000,523.080000,0,0.000000,0.008403,0.000000,0.000000,0.001681,0.000000,0.000000,0.000000,0.008403,0.000000,1
4,251123,45589200,2,2,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,0.456549,12.333333,53.766640,0,0.000000,0.204918,0.000000,0.000000,0.049180,0.000000,0.008197,0.057377,0.180328,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488872,535266,38983571,2,2,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.300000,0.422837,8.714286,56.783167,0,0.008000,0.064000,0.000000,0.012000,0.006400,0.000000,0.008000,0.000000,0.048000,195.000000,1
488873,36706,53264639,1,1,0.017241,0.000,0.00,0,0.0,0.0,0.000000,0.012195,0.668494,35.662921,69.031216,2,0.051724,2.043103,0.017241,0.301724,0.339655,0.068966,0.922414,0.137931,1.172414,188.742857,4
488874,31570,44693578,1,1,0.229508,1.625,2.25,0,0.0,0.0,0.000000,0.094340,0.579213,13.444444,50.611380,0,0.016393,0.409836,0.000000,0.000000,0.118033,0.000000,0.049180,0.237705,0.303279,184.880000,4
488875,239209,42760145,2,1,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,0.221303,9.578947,131.188182,0,0.000000,0.113821,0.000000,0.000000,0.030894,0.000000,0.024390,0.130081,0.000000,0.000000,1


In [5]:
train = pd.get_dummies(train, columns=['tier','gender'])
test = pd.get_dummies(test, columns=['tier','gender'])

train['age_group'] = train['age_group']-1

feature_cols = [col for col in train.columns.tolist() if col not in ['age_group']]
target_cols = ['age_group']

# KFOLD
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

for i, (trn, val) in enumerate(skf.split(train[target_cols], train[target_cols])):
    train.loc[val, 'kfold'] = i
train['kfold'] = train['kfold'].astype(int)

In [6]:
# Reporting util for different optimizers
def report_perf(optimizer, X, y, title, callbacks=None):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time.time()
    if callbacks:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_
    print(best_params)
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time.time() - start, 
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))    
    print('Best parameters:')
    print(best_params)
    print()
    return best_params

In [7]:
import lightgbm as lgb

class DoubleLGB():
    def __init__(self):
        self.mod1 = lgb.LGBMClassifier(max_depth=-1,
                                   n_estimators=200,
                                   device='gpu',
                                   silent=True, 
                                   metric='None')
        self.mod2 = lgb.LGBMClassifier(max_depth=-1,
                                   n_estimators=200,
                                   device='gpu',
                                   silent=True, 
                                   metric='None')
        
    def fit(self, x, y):
        self.mod1.fit(x, y)
        self.mod2.fit(x, y)
        
    def predict_proba(self, x):
        pred1 = self.mod1.predict_proba(x)
        pred2 = self.mod2.predict_proba(x)
        
        return (pred1+pred2)/2
    
    def predict(self, x):
        pred1 = self.mod1.predict_proba(x)
        pred2 = self.mod2.predict_proba(x)
        
        pred = (pred1+pred2)/2
        
        pred_classes = np.argmax(pred, axis=1)
        return pred_classes

In [8]:
import lightgbm as lgb

def run_training():
    oof = np.zeros((train.shape[0], 4))
    pred = np.zeros((test.shape[0], 4))
    
    for fold in range(5):
        print(f"\nStarting FOLD: {fold}")
        start = time.time()
        

        trn_idx = train['kfold'] != fold
        val_idx = train['kfold'] == fold
        trn = train.loc[trn_idx, :]
        val = train.loc[val_idx, :]

        xtr, ytr = trn[feature_cols].values, trn[target_cols].values
        xval, yval = val[feature_cols].values, val[target_cols].values
        xtest = test[feature_cols].values

        model = lgb.LGBMClassifier(num_leaves=50, reg_alpha=0.01)

        print("fitting model...")                           
        model.fit(xtr, ytr.reshape(-1,))
        print("calculating training preds...")
        train_preds = model.predict(xtr)
        print("calculating validation preds...")
        val_preds = model.predict(xval)
        print("Training Accuracy Score - ", accuracy_score(ytr, train_preds))
        print("Training F1 Score - ", f1_score(ytr, train_preds, average='weighted'))
        
        print("Validation Accuracy Score - ", accuracy_score(yval, val_preds))
        print("Validation F1 Score - ", f1_score(yval, val_preds, average='weighted'))
        
        oof[val_idx, :] += model.predict_proba(xval)
        
        # Test preds
        pred += model.predict_proba(xtest)/5
        
        print(f"FOLD {fold} completed in {time.time()-start} seconds")
        
    return oof, pred

In [9]:
oof_lgb, pred_lgb = run_training() 


Starting FOLD: 0
fitting model...
calculating training preds...
calculating validation preds...
Training Accuracy Score -  0.7913607993843023
Training F1 Score -  0.7873003642298387
Validation Accuracy Score -  0.7733186057928326
Validation F1 Score -  0.769280830969272
FOLD 0 completed in 42.026206970214844 seconds

Starting FOLD: 1
fitting model...
calculating training preds...
calculating validation preds...
Training Accuracy Score -  0.790309919943953
Training F1 Score -  0.7859634575322365
Validation Accuracy Score -  0.7747197676321388
Validation F1 Score -  0.7702082698025807
FOLD 1 completed in 40.89071345329285 seconds

Starting FOLD: 2
fitting model...
calculating training preds...
calculating validation preds...
Training Accuracy Score -  0.7900598820767983
Training F1 Score -  0.7858423450681611
Validation Accuracy Score -  0.7748811045768346
Validation F1 Score -  0.7702496070783655
FOLD 2 completed in 39.02066516876221 seconds

Starting FOLD: 3
fitting model...
calculati

In [10]:
def find_oof_score(oof):
    predictions = oof.argmax(axis=1)
    print("OOF Accuracy Score - ", accuracy_score(train[target_cols], predictions))
    print("OOF F1 Score - ", f1_score(train[target_cols], predictions, average='weighted'))
    
# without class weights - 7475

In [11]:
find_oof_score(oof_lgb)

OOF Accuracy Score -  0.7739390480632142
OOF F1 Score -  0.7697550527301407


In [12]:
np.save('oof_lgb.npy', oof_lgb)
np.save('pred_lgb.npy', pred_lgb)